In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
import gezi
from gezi import tqdm
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [56]:
df = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [57]:
train_names, train_texts = [], []
for file in tqdm(glob.glob('../input/feedback-prize-2021/train/*.txt')):
    train_names.append(os.path.basename(file).replace('.txt', ''))
    train_texts.append(open(file, 'r').read())
train_texts = pd.DataFrame({'id': train_names, 'text': train_texts})

  0%|          | 0/15594 [00:00<?, ?it/s]

In [58]:
train = pd.merge(df, train_texts, how = 'left', on = 'id')
train["essay_len"] = train["text"].apply(lambda x: len(x))

In [60]:
x = train[train.id=='7330313ED3F0']

In [61]:
x

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,text,essay_len
12917,7330313ED3F0,1623268961537,0,77,"Around the world, citites are getting improved...",Position,Position 1,0 1 2 3 4 5 6 7 8 9 10 11 12,"Around the world, citites are getting improved...",3846
12918,7330313ED3F0,1623268987144,77,455,With the new ways of transportation such as b...,Evidence,Evidence 1,13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 2...,"Around the world, citites are getting improved...",3846
12919,7330313ED3F0,1623269491488,456,1014,"In Vauban, Germany many residents of this comm...",Evidence,Evidence 2,75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 9...,"Around the world, citites are getting improved...",3846
12920,7330313ED3F0,1623269485864,1015,1130,"The idea of a ""car reduced"" community is very ...",Claim,Claim 1,183 184 185 186 187 188 189 190 191 192 193 19...,"Around the world, citites are getting improved...",3846
12921,7330313ED3F0,1623269552585,3377,3846,Many citites in the world are starting to see ...,Concluding Statement,Concluding Statement 1,584 585 586 587 588 589 590 591 592 593 594 59...,"Around the world, citites are getting improved...",3846


In [62]:
x.text.values[0]

'Around the world, citites are getting improved by reducing the usage of cars. With the new ways of transportation such as bike, foot, car-pool and public transportation (bus, train, taxi) there has been many great out comes. Many people should join in on this limiting of car usage. It saves time, conserves resources, lowers emmisions and improves safety. Many citites, such as Vauban, Paris and Bogota have taken steps to creating a car free community.\n\nIn Vauban, Germany many residents of this community dont even own a car. The people are allowed to own a car but they have two options of parking that result in buying a space or a home with a space. With this great cost of spaces, up to 70 percent of the families do not own a car. Many people living this life style are very happy with not owning a car. Heidrun Walter thought that "when [she] had a car [she] was always tense. [And she] is much happier this way". It was proven that 12 percent of greenhouse gas emissions in Europe is fro

In [63]:
list(x.discourse_text.values)

['Around the world, citites are getting improved by reducing the usage of cars.',
 ' With the new ways of transportation such as bike, foot, car-pool and public transportation (bus, train, taxi) there has been many great out comes. Many people should join in on this limiting of car usage. It saves time, conserves resources, lowers emmisions and improves safety. Many citites, such as Vauban, Paris and Bogota have taken steps to creating a car free community. ',
 'In Vauban, Germany many residents of this community dont even own a car. The people are allowed to own a car but they have two options of parking that result in buying a space or a home with a space. With this great cost of spaces, up to 70 percent of the families do not own a car. Many people living this life style are very happy with not owning a car. Heidrun Walter thought that "when [she] had a car [she] was always tense. [And she] is much happier this way". It was proven that 12 percent of greenhouse gas emissions in Europ

In [64]:
[x.text.values[0].find(a) for a in x.discourse_text]

[0, -1, 456, 1015, 3377]

In [68]:
[x.text.values[0][int(a):int(b)] for a, b in zip(x.discourse_start, x.discourse_end)]

['Around the world, citites are getting improved by reducing the usage of cars.',
 ' With the new ways of transportation such as bike, foot, car-pool and public transportation (bus, train, taxi) there has been many great out comes. Many people should join in on this limiting of car usage. It saves time, conserves resources, lowers emmisions and improves safety. Many citites, such as Vauban, Paris and Bogota have taken steps to creating a car free community.\n',
 'In Vauban, Germany many residents of this community dont even own a car. The people are allowed to own a car but they have two options of parking that result in buying a space or a home with a space. With this great cost of spaces, up to 70 percent of the families do not own a car. Many people living this life style are very happy with not owning a car. Heidrun Walter thought that "when [she] had a car [she] was always tense. [And she] is much happier this way". It was proven that 12 percent of greenhouse gas emissions in Euro

In [72]:
texts = x.text.values[0].split()
for pred in x.predictionstring:
  l = pred.split()
  s, e = int(l[0]), int(l[-1])
  print('----', '-'.join(texts[s:e+1]), '###')

---- Around-the-world,-citites-are-getting-improved-by-reducing-the-usage-of-cars. ###
---- With-the-new-ways-of-transportation-such-as-bike,-foot,-car-pool-and-public-transportation-(bus,-train,-taxi)-there-has-been-many-great-out-comes.-Many-people-should-join-in-on-this-limiting-of-car-usage.-It-saves-time,-conserves-resources,-lowers-emmisions-and-improves-safety.-Many-citites,-such-as-Vauban,-Paris-and-Bogota-have-taken-steps-to-creating-a-car-free-community. ###
---- In-Vauban,-Germany-many-residents-of-this-community-dont-even-own-a-car.-The-people-are-allowed-to-own-a-car-but-they-have-two-options-of-parking-that-result-in-buying-a-space-or-a-home-with-a-space.-With-this-great-cost-of-spaces,-up-to-70-percent-of-the-families-do-not-own-a-car.-Many-people-living-this-life-style-are-very-happy-with-not-owning-a-car.-Heidrun-Walter-thought-that-"when-[she]-had-a-car-[she]-was-always-tense.-[And-she]-is-much-happier-this-way".-It-was-proven-that-12-percent-of-greenhouse-gas-emissio

In [ ]:
## 由此可以看出precitionstring是按照 .split()来切割word计数的， 因此等于去除了\n 暂时可以先这么弄 后续考虑添加\n信息回去

In [74]:
'a b\n c'.split()

['a', 'b', 'c']

In [73]:
texts

['Around',
 'the',
 'world,',
 'citites',
 'are',
 'getting',
 'improved',
 'by',
 'reducing',
 'the',
 'usage',
 'of',
 'cars.',
 'With',
 'the',
 'new',
 'ways',
 'of',
 'transportation',
 'such',
 'as',
 'bike,',
 'foot,',
 'car-pool',
 'and',
 'public',
 'transportation',
 '(bus,',
 'train,',
 'taxi)',
 'there',
 'has',
 'been',
 'many',
 'great',
 'out',
 'comes.',
 'Many',
 'people',
 'should',
 'join',
 'in',
 'on',
 'this',
 'limiting',
 'of',
 'car',
 'usage.',
 'It',
 'saves',
 'time,',
 'conserves',
 'resources,',
 'lowers',
 'emmisions',
 'and',
 'improves',
 'safety.',
 'Many',
 'citites,',
 'such',
 'as',
 'Vauban,',
 'Paris',
 'and',
 'Bogota',
 'have',
 'taken',
 'steps',
 'to',
 'creating',
 'a',
 'car',
 'free',
 'community.',
 'In',
 'Vauban,',
 'Germany',
 'many',
 'residents',
 'of',
 'this',
 'community',
 'dont',
 'even',
 'own',
 'a',
 'car.',
 'The',
 'people',
 'are',
 'allowed',
 'to',
 'own',
 'a',
 'car',
 'but',
 'they',
 'have',
 'two',
 'options',
 'of',

In [50]:
dis_starts, dis_ends = [], []
starts, ends = [], []
# text_list = []
for row in tqdm(train.itertuples(), total=len(train)):
  preds = row.predictionstring.split()
  s, e = int(preds[0]), int(preds[-1])
  # texts = row.text.split(' ')
  # start = len(' '.join(texts[:s]))
  # end = len(' '.join(texts[:e + 1])) 
  print('-------------', row.text)
  print('#############', row.discourse_text)
  start = row.text.index(row.discourse_text)
  print(start)
  end = start + len(row.discourse_text)
  dis_starts.append(start)
  dis_ends.append(end)
  starts.append(s)
  ends.append(e + 1)
  # text = ' '.join(texts)
  # text_list.append(text)
  
train['discourse_start'], train['discourse_end'] = dis_starts, dis_ends
train['start'], train['end'] = starts, ends
# train['text'] = text_list

  0%|          | 0/144293 [00:00<?, ?it/s]

------------- Phones

Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.

When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.

Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing 

ValueError: substring not found

In [ ]:
#initialize columns
train['gap_before'] = False
train['gap_length'] = np.nan

#set the first one
train.loc[0, 'gap_before'] = True
train.loc[0, 'gap_length'] = 8

#loop over rest
for i in tqdm(range(1, len(train))):
    #gap if difference is not 1 within an essay
    if ((train.loc[i, "id"] == train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] > 0)):
        train.loc[i, 'gap_before'] = True
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"]
    #gap if the first discourse of an new essay does not start at 0
    elif ((train.loc[i, "id"] != train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] != 0)):
        train.loc[i, 'gap_before'] = True
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"]

train['gap_end_length'] = train.essay_len - train.discourse_end

In [ ]:
def add_gap_rows(essay):
    cols_to_keep = ['discourse_start', 'discourse_end', 'discourse_type', 'gap_length', 'gap_end_length', 'text']
    df_essay = train.query('id == @essay')[cols_to_keep].reset_index(drop = True)

    #index new row
    insert_row = len(df_essay)

    #add gaps in between
    for i in range(len(df_essay)):
        if df_essay.loc[i,"gap_length"] >0:
            start = df_essay.loc[i-1, "discourse_end"] 
            end = df_essay.loc[i, 'discourse_start'] 
            text = df_essay.loc[i, 'text']
            disc_type = "Nothing"
            gap_end = np.nan
            gap = np.nan
            df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end, text]
            insert_row += 1

    df_essay = df_essay.sort_values(by = "discourse_start").reset_index(drop=True)

    #add gap at end
    if df_essay.loc[(len(df_essay)-1),'gap_end_length'] > 0:
        start = df_essay.loc[(len(df_essay)-1), "discourse_end"] 
        end = start + df_essay.loc[(len(df_essay)-1), 'gap_end_length']
        disc_type = "Nothing"
        gap_end = np.nan
        gap = np.nan
        text = df_essay.loc[(len(df_essay)-1), 'text']
        df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end, text]
        
    return(df_essay)

In [ ]:
def print_colored_essay(essay):
    df_essay = add_gap_rows(essay)

    ents = []
    for i, row in df_essay.iterrows():
        ents.append({
                        'start': int(row['discourse_start']), 
                        'end': int(row['discourse_end']), 
                        'label': row['discourse_type']
                    })

    data = row['text']

    doc2 = {
        "text": data,
        "ents": ents,
    }

    colors = {'Lead': '#EE11D0','Position': '#AB4DE1','Claim': '#1EDE71','Evidence': '#33FAFA','Counterclaim': '#4253C1','Concluding Statement': 'yellow','Rebuttal': 'red'}
    options = {"ents": df_essay.discourse_type.unique().tolist(), "colors": colors}
    spacy.displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True);

In [ ]:
add_gap_rows("129497C3E0FC")

In [ ]:
print_colored_essay("129497C3E0FC")

In [ ]:
print_colored_essay("7330313ED3F0")

In [ ]:
list(train[train.id=='7330313ED3F0'].discourse_text.values)